### **利用gdal开源库进行栅格文件处理**
#### 1. 投影转换
#### 2. 重采样


In [1]:
from osgeo import gdal
from osgeo import osr


#### 1. 投影转换

In [ ]:
path_img = 'data/Section-2/s2_chenggong_20200411_6bands_20m.tif'
path_reproj = 'data/Section-5/s2_chenggong_20200411_6bands_20m_wgs84.tif'

In [3]:
## 读入栅格数据，包括影像信息和地理信息
dset = gdal.Open(path_img)
proj = dset.GetProjection()    ### 获取影像投影
print('projection:', proj)


projection: PROJCS["WGS 84 / UTM zone 47N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",99],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32647"]]


In [4]:
### 左上角utm坐标计算, 及utm投影下影像范围
utm47_epsg = 32647
wgs84_epsg = 4326
utm47 = osr.SpatialReference(); 
utm47.ImportFromEPSG(utm47_epsg)
wgs84 = osr.SpatialReference(); 
wgs84.ImportFromEPSG(wgs84_epsg)
proj_utm47_wkt = utm47.ExportToWkt()
proj_wgs84_wkt = wgs84.ExportToWkt()
wgs84.SetAxisMappingStrategy(osr.OAMS_TRADITIONAL_GIS_ORDER)
transform = osr.CoordinateTransformation(utm47, wgs84)
geo_trans = dset.GetGeoTransform()
print(geo_trans[0], geo_trans[3])   ## 左上角x,y坐标
(x_min_wgs84, y_max_wgs84, z) = transform.TransformPoint(geo_trans[0], geo_trans[3], 0)  ## 输入参数为(纬度，经度）,输出参数为（x，y），注意顺序
print('the left-up coordinate of the utm projection:', x_min_wgs84, y_max_wgs84)


874420.0 2769600.0
the left-up coordinate of the utm projection: 102.70864034390206 24.995729297438114


In [5]:
### 像元尺寸（分辨率）计算
x_max_utm = geo_trans[0]+geo_trans[1]*dset.RasterXSize  ## x方向最大值
y_min_utm = geo_trans[3]+geo_trans[5]*dset.RasterYSize  ## y方向最小值
(x_max_wgs84, y_min_wgs84, z) = transform.TransformPoint(x_max_utm, y_min_utm, 0)  ## 输入参数为(纬度，经度, 输出参数为（x，y），注意顺序
print(x_max_wgs84, y_min_wgs84)
width_pixel = (x_max_wgs84-x_min_wgs84)/dset.RasterXSize
height_pixel = (y_max_wgs84-y_min_wgs84)/dset.RasterYSize
print(width_pixel, height_pixel)


102.99381089981256 24.694382912987106
0.0001913896348392598 0.00018487508248528128


In [6]:
### 定义新的转换参数和投影
geotrans_wgs84 = (102.70864034390206, 0.0001913896348392598, 0.0, 24.995729297438114, 0, -0.00018487508248528128)


In [7]:
driver = gdal.GetDriverByName('GTiff')
dset_wgs84 = driver.Create(path_reproj, xsize=dset.RasterXSize, \
                                ysize=dset.RasterYSize, bands=dset.RasterCount, eType=gdal.GDT_Int16)
dset_wgs84.SetGeoTransform(geotrans_wgs84)
dset_wgs84.SetProjection(proj_wgs84_wkt)

## 影像重投影
reproj = gdal.ReprojectImage(src_ds=dset, dst_ds=dset_wgs84, src_wkt=proj_utm47_wkt, dst_wkt=proj_wgs84_wkt)
dset_wgs84 = None  ## 关闭文件



### 重采样

In [8]:
path_resam = 'data/Section-5/s2_chenggong_20200411_6bands_20m_resam.tif'
res=60  ## 定义分辨率


In [9]:
dset = gdal.Open(path_img)
geo_trans = dset.GetGeoTransform()
x_min, y_max = geo_trans[0], geo_trans[3]
x_max = x_min + geo_trans[1]*dset.RasterXSize
y_min = y_max + geo_trans[5]*dset.RasterYSize
print((x_min,y_max), (x_max, y_min))


(874420.0, 2769600.0) (904220.0, 2737000.0)


In [10]:
### 1.计算影像尺寸
x_size = (x_max - x_min)/res  ## x方向尺寸
y_size = (y_max - y_min)/res   ## y方向尺寸
print(x_size, y_size)
x_size, y_size = int(x_size), int(y_size)
### 2.更新影像分辨率
res_x = (x_max - x_min)/x_size  ##  新的x方向分辨率（像元宽）
res_y = (y_max - y_min)/y_size  ##  新的y方向分辨率（像元高）
print('Final image size (x, y):', int(x_size), int(y_size))
print('New resolution (x, y):', res_x, res_y)
### 3. 定义地理转换参数
geotrans_resample = (874420.0, 60.08064516129032, 0, 2769600.0, 0, -60.03683241252302)


496.6666666666667 543.3333333333334
Final image size (x, y): 496 543
New resolution (x, y): 60.08064516129032 60.03683241252302


In [11]:
driver = gdal.GetDriverByName('GTiff')
dset_resam = driver.Create(path_resam, xsize=x_size, \
                                ysize=y_size, bands=dset.RasterCount, eType=gdal.GDT_Int16)
dset_resam.SetGeoTransform(geotrans_resample)
dset_resam.SetProjection(dset.GetProjection())
## 影像重投影
reproj = gdal.ReprojectImage(src_ds=dset, dst_ds=dset_resam, eResampleAlg=gdal.GRA_Bilinear)
dset_resam = None  ## 关闭文件


### 快捷方式
主要函数：gdal.Warp()/gdalwarp   
参考：   
1.https://gdal.org/api/python/osgeo.gdal.html.   
2.https://gdal.org/programs/gdalwarp.html

In [12]:
path_img = 'data/Section-2/s2_chenggong_20200411_6bands_20m.tif'
path_reproj = 'data/Section-5/s2_chenggong_20200411_6bands_20m_wgs84_2.tif'
path_resam = 'data/Section-5/s2_chenggong_20200411_6bands_20m_resam_2.tif'

### 重投影
warp_reproj = gdal.Warp(destNameOrDestDS=path_reproj, srcDSOrSrcDSTab=path_img, dstSRS='EPSG:4326')  ### 利用gdal.Warp()进行投影转换
warp_reproj = None   ### !!关闭文件


In [14]:
### 重采样
warp_resam = gdal.Warp(destNameOrDestDS=path_resam, srcDSOrSrcDSTab=path_img, xRes=60, yRes=60)  ### 利用gdal.Warp()进行投影转换
# warp_resam = gdal.Warp(destNameOrDestDS=path_resam, srcDSOrSrcDSTab=path_img, xRes=60, yRes=60, resampleAlg=gdal.GRA_Bilinear)  ### 利用gdal.Warp()进行投影转换
warp_resam = None   ### !!关闭文件

